## Домашняя работа №5

<ol>
    <li>Попробуйте изменить параметры нейронной сети работающей с датасетом imdb либо нейронной сети работающей airline-passengers(она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить ее точность. Приложите анализ.</li>
    <li>Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.</li>
    <li>* Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM</li>
    <li>* Предложите свои варианты решения проблемы исчезающего градиента в RNN</li>
</ol>

In [19]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

np.random.seed(0)

max_features = 25000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 120
batch_size = 50 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 120)
x_test shape: (25000, 120)
Построение модели...
Процесс обучения...


C:\Users\fini1\anaconda3\envs\ml_env\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 316s 13ms/step - loss: 0.4582 - accuracy: 0.7862 - val_loss: 0.3425 - val_accuracy: 0.8567
Epoch 2/3
25000/25000 [==============================] - 342s 14ms/step - loss: 0.3117 - accuracy: 0.8746 - val_loss: 0.3792 - val_accuracy: 0.8373
Epoch 3/3
25000/25000 [==============================] - 43s 2ms/step
Результат при тестировании: 0.34804962868988515
Тестовая точность: 0.85944002866745


In [20]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 512, 512
NUM_ITERATIONS = 5 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в 
вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на 
предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print(" ")
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


 
Итерация #: 0
Epoch 1/3
158773/158773 [==============================] - 347s 2ms/step - loss: 2.4728
Epoch 2/3
158773/158773 [==============================] - 344s 2ms/step - loss: 1.9474
Epoch 3/3
158773/158773 [==============================] - 368s 2ms/step - loss: 1.7517
Генерация из посева: e problem.
e problem. there was to doon a little said a in the mouse, a dont the mouse, a dont the mouse, a dont the mous 
Итерация #: 1
Epoch 1/3
158773/158773 [==============================] - 361s 2ms/step - loss: 1.6121
Epoch 2/3
158773/158773 [==============================] - 357s 2ms/step - loss: 1.5009
Epoch 3/3
158773/158773 [==============================] - 346s 2ms/step - loss: 1.4088
Генерация из посева: use did no
use did not any gryphon. alice and great curious to be an the project gutenberg-tm electronic works in that th 
Итерация #: 2
Epoch 1/3
158773/158773 [==============================] - 358s 2ms/step - loss: 1.3302
Epoch 2/3
158773/158773 [===========================

## Выводы

1.

Лучший результат "Тестовая точность: 0.85944002866745" был получен при настройках сети:

max_features = 25000, maxlen = 120, batch_size = 50, optimizer='RMSprop', loss='binary_crossentropy', epochs=3

Что интересно, при данной конфигурации увеличение количества epochs больше 3 приводило к переобучению, изменение max_features в большую или меньшу сторону, также снижало тестовую точность. По всей видимости, такие изменения ухудшают последовательность обрабатываемой информации. Возможно к улучшению результата могло бы привести увелечение количества слоев для сохранения точности последовательности.
Изменение других настроек не приводило к существенным улучшениям или ухудшениям тестовой точности сети, кроме изменений функции потерь, в некоторых случаях результаты сильно ухудшались.


2.

Первое, что хочу отметить, это важность подготовки входных текстовых данных, а именно отстуствие слов не имеющих прямого отношения к контексту исходного материала, например, как в нашем случае, gutenberg-tm electronics.
Основные параметры, которые я подверг изменению:

BATCH_SIZE, HIDDEN_SIZE = 512, 512
NUM_ITERATIONS = 5 
NUM_EPOCHS_PER_ITERATION = 3

По результатам мы видим, что данные настройки позволяют генерировать разнообразный текст, что говорит о формировании вполне адекватных последовательностей и зависимостей слов. Учитывая, что функция потерь по мере обучения продолжала уверенно снижаться можно предположить, что при данных настройках остался потенциал для улучшения формирования последовательностей генерации слов представленной сетью. И вполне очевидно, что для генерации разнообразного адекватного текста при данной конфигурации достачно NUM_ITERATIONS = 3.



P.S. Весьма не просто комментировать работу чёрных ящиков :)